# Model Quality Checks

In [1]:
import pandas as pd
import numpy as np

from sklearn import datasets, ensemble
from utils import feature_engineering

from evidently.test_suite import TestSuite
from evidently.test_preset import NoTargetPerformanceTestPreset, BinaryClassificationTestPreset, DataDriftTestPreset

## Load data

In [2]:
bank_marketing = datasets.fetch_openml(name="bank-marketing", as_frame="auto")
bank_marketing_data = bank_marketing.frame

D:\Programming Train\ml_monitoring\venv\Lib\site-packages\sklearn\datasets\_openml.py:311: UserWarning: Multiple active versions of the dataset matching the name bank-marketing exist. Versions may be fundamentally different, returning version 1.
  warn(
D:\Programming Train\ml_monitoring\venv\Lib\site-packages\sklearn\datasets\_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
processed_data = feature_engineering(bank_marketing_data)

processed_data_train = processed_data[:5000]
processed_data_reference = processed_data[5000:7000]
processed_data_prod_simulation = processed_data[7000:]
batch_size = 2000

In [4]:
processed_data_train.shape

(5000, 49)

## Load model

In [5]:
import pickle
with open("rf_model.pckl", "rb") as f:
    model = pickle.load(f)

In [6]:
processed_data_reference["prediction"] = model.predict(processed_data_reference.iloc[:, :-1])
processed_data_prod_simulation["prediction"] = model.predict(processed_data_prod_simulation.iloc[:, :-1])

## Model output checks

In [7]:
no_target_performance_suite = TestSuite(tests=[NoTargetPerformanceTestPreset()])
no_target_performance_suite.run(reference_data=processed_data_reference, current_data=processed_data_prod_simulation[:batch_size])
no_target_performance_suite.show(mode="auto")

In [8]:
no_target_performance_suite = TestSuite(tests=[NoTargetPerformanceTestPreset()])
no_target_performance_suite.run(reference_data=processed_data_reference, current_data=processed_data_prod_simulation[batch_size:2 * batch_size])
no_target_performance_suite.show(mode="auto")

## Model quality checks

In [9]:
model_performance_suite = TestSuite(tests=[BinaryClassificationTestPreset()])
model_performance_suite.run(reference_data=processed_data_reference, current_data=processed_data_prod_simulation[batch_size:2 * batch_size])
model_performance_suite.show(mode="auto")